Load dataset and preprocess

In [149]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
# Ensure data and dimensions are correct
assert (x_train.shape == (50000, 32, 32, 3))
assert (y_train.shape == (50000, 1))
assert (x_test.shape == (10000, 32, 32, 3))
assert (y_test.shape == (10000,1))

# Preprocessing
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# flatten x_train and x_test arrays
# New layout (50000, 32x32x3)
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# One-hot encode y_train and y_test
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [150]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

class Sigmoid:
    def forward(self, x):
        # x can be any shape
        return sigmoid(x)

    def backward(self, a):
        # d/dx sigmoid(x) = s * (1 - s)
        s = sigmoid(a)
        return s * (1.0 - s)

class ReLU:
    def forward(self, x):
        return np.maximum(0.0, x)

    def backward(self, x):
        return (x > 0).astype(float)
    
class LeakyReLU:
    def __init__(self, a=0.01):
        # hyperparam a value, so we can tune it for testing
        self.a = a

    def forward(self, x):
        return np.maximum(x, self.a * x)
    
    def backward(self, x):
        return np.where(x > 0, 1.0, self.a)

In [151]:
class Softmax:
    def __init__(self):
        self.y_pred = None

    def forward(self, x):
        #LSE trick
        x_stable = np.exp(x - np.max(x, axis = 1, keepdims=True)) #If x is a batch (in the shape of: [batch_size, num_classes]), then np.max(x) would only return a single scalar value, which is the same value subtracted from all rows
        exp_sum = np.sum(x_stable, axis=1, keepdims=True)
        self.y_pred = x_stable / exp_sum
        return self.y_pred

    # Simplified dCE_Loss/dSoftMax 
    def backward(self, y_actual):
        return self.y_pred - y_actual

In [152]:
class Dropout:
    def __init__ (self, drop_rate):
        # Noting that the drop_rate resembles the probability that a unit will be set to 0. (e.g. 0.5 for 50%)
        self.drop_rate = drop_rate
        # Mask will store the indices of the units which are kept (set to 1) during training.
        self.mask = None


    def forward(self, x, training=True):
        """
        Performs the forward pass for the dropout layer

        args:
            x (np.array) is the input data
            training (bool) if true, apply dropout, else return input as is.
        
        returns:
            The output data after applying dropout (if training is set to True), or data is just passing through.
        """
        # Mask is created, 1 (True) to keep the unit at 0 (False) to drop it.
        # (1-drop_rate) is the probability of keeping a unit.
        if training:
            self.mask = np.random.rand(*x.shape) > self.drop_rate
            # Multiply the mask by 1/p to maintain the expected value of values.
            return x * self.mask*1/(1-self.drop_rate)
        else:
            return x
    
    def backward(self, d_out):
        """
        Performs the backwards pass for the dropout layer.

        args:
            d_out (np.array) is the gradient from the subsequent layer

        returns:
            np.ndarray is the gradient passed to the preceding layer
        """
        # The gradient only flows through the neurons that weren't dropped in the forward pass, and the same inverted scaling factor
        # 1/(1-drop_rate) is applied to the gradient.
        return d_out * self.mask * 1/(1-self.drop_rate)

Input layer: 3072 nodes<br>
Output classification layer: 10 nodes<br><br>To do:<br> Implement L1/L2 Regularisers<br>Look at bottom where training and testing is happening, Add metrics for analysis (for graphs)<br>Add Comments

In [153]:
class TrainingMonitor:
    """
    Callback-style monitor for tracking model performance during training.
    """
    
    def __init__(self, validation_x=None, validation_y=None):
        self.validation_x = validation_x
        self.validation_y = validation_y
        
        # Metric storage using lists
        self.metrics = {
            'training': {'loss': [], 'accuracy': []},
            'validation': {'loss': [], 'accuracy': []}
        }
    
    def has_validation_data(self):
        """Check if validation dataset is configured."""
        return self.validation_x is not None and self.validation_y is not None
    
    def record_epoch(self, epoch_number, loss_value, accuracy_value, phase='training'):
        """
        Store metrics for a completed epoch.
        
        Args:
            epoch_number (int): Current epoch index
            loss_value (float): Computed loss
            accuracy_value (float): Computed accuracy
            phase (str): Either 'training' or 'validation'
        """
        self.metrics[phase]['loss'].append(loss_value)
        self.metrics[phase]['accuracy'].append(accuracy_value)
    
    def log_progress(self, current_epoch, total_epochs):
        """Print formatted progress update."""
        train_loss = self.metrics['training']['loss'][-1]
        train_acc = self.metrics['training']['accuracy'][-1]
        
        output = f"Epoch [{current_epoch + 1}/{total_epochs}] | "
        output += f"Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}"
        
        if self.has_validation_data():
            val_loss = self.metrics['validation']['loss'][-1]
            val_acc = self.metrics['validation']['accuracy'][-1]
            output += f" | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
        
        print(output)
    
    def visualize_performance(self):
        """Generate performance visualization plots."""
        import matplotlib.pyplot as plt
        
        num_epochs = len(self.metrics['training']['loss'])
        epoch_numbers = np.arange(1, num_epochs + 1)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss visualization
        ax1.plot(epoch_numbers, self.metrics['training']['loss'], 
                marker='o', markersize=3, linewidth=2.5, color='#2E86AB', label='Training')
        if self.has_validation_data():
            ax1.plot(epoch_numbers, self.metrics['validation']['loss'], 
                    marker='s', markersize=3, linewidth=2.5, color='#E63946', label='Validation')
        ax1.set_xlabel('Epoch Number', fontsize=13)
        ax1.set_ylabel('Cross-Entropy Loss', fontsize=13)
        ax1.set_title('Loss Progression', fontsize=15, fontweight='bold', pad=15)
        ax1.legend(loc='best', framealpha=0.9)
        ax1.grid(True, linestyle='--', alpha=0.4)
        
        # Accuracy visualization
        ax2.plot(epoch_numbers, self.metrics['training']['accuracy'], 
                marker='o', markersize=3, linewidth=2.5, color='#2E86AB', label='Training')
        if self.has_validation_data():
            ax2.plot(epoch_numbers, self.metrics['validation']['accuracy'], 
                    marker='s', markersize=3, linewidth=2.5, color='#E63946', label='Validation')
        ax2.set_xlabel('Epoch Number', fontsize=13)
        ax2.set_ylabel('Classification Accuracy', fontsize=13)
        ax2.set_title('Accuracy Progression', fontsize=15, fontweight='bold', pad=15)
        ax2.legend(loc='best', framealpha=0.9)
        ax2.grid(True, linestyle='--', alpha=0.4)
        
        plt.tight_layout()
        plt.show()


def create_data_splits(x_full, y_full, train_size, val_size):
    """
    Utility function to partition dataset into train/validation subsets.
    
    Args:
        x_full (np.ndarray): Complete feature dataset
        y_full (np.ndarray): Complete label dataset
        train_size (int): Number of training samples
        val_size (int): Number of validation samples
        
    Returns:
        tuple: (x_train, y_train, x_val, y_val)
    """
    x_train = x_full[:train_size]
    y_train = y_full[:train_size]
    
    x_val = x_full[train_size:train_size + val_size]
    y_val = y_full[train_size:train_size + val_size]
    
    return x_train, y_train, x_val, y_val

In [154]:
class NeuralNetwork:
    
    def __init__(self, x_train, y_train, hidden_layers, hidden_layer_sizes, activation_func, optimiser, learning_rate=0.03, dropout_rate=0.5):
        
        self.inputs = x_train
        self.outputs = y_train

        self.hidden_layers = hidden_layers

        if (len(hidden_layer_sizes) != hidden_layers):
            raise ValueError("Neurons array length mismatch with hidden layers amount")
        self.hidden_layers_sizes = hidden_layer_sizes

        if not isinstance(activation_func, (Sigmoid, ReLU, LeakyReLU)):
            raise TypeError("Activation function must be of type Sigmoid, ReLU or LeakyReLU")
        self.activation_func = activation_func

        self.learning_rate = learning_rate

        self.weights, self.biases = self.initalise_parameters()

        self.activations = [None] * (self.hidden_layers + 2)

        # Dropout instance per hidden layer
        self.dropout_layers = [Dropout(dropout_rate) for _ in range(self.hidden_layers)]
        
        # Might remove - recheck
        self.dropout_outputs = [None] * self.hidden_layers

        self.softmax = Softmax()

        # For Optimisers
        self.optimiser = optimiser    
        # RMS_Prop
        if optimiser == "RMS_Prop":
            self.r_weights, self.r_biases = self.create_zeroed_matrix_per_parameter()
        # Adam
        if optimiser == "Adam":
            self.adam_moment1_weights, self.adam_moment1_biases = self.create_zeroed_matrix_per_parameter()
            self.adam_moment2_weights, self.adam_moment2_biases = self.create_zeroed_matrix_per_parameter()
            self.adam_timestep = 0


    def initalise_parameters(self):
        network_layout = [len(self.inputs[0])]
        for i in range(self.hidden_layers):
            network_layout.append(self.hidden_layers_sizes[i])
        network_layout.append(len(self.outputs[0]))
        
        weights = []
        biases = []

        # Sigmoid weight initialisation
        if isinstance(self.activation_func, Sigmoid):
            for i in range(len(network_layout) -  1):
                input = network_layout[i]
                output = network_layout[i+1]
                weight_init = (np.random.uniform(low=-1, high=1, size=(output, input))) / (np.sqrt(input))
                bias = np.zeros(output)
                weights.append(weight_init)
                biases.append(bias)
        
        # ReLU weight initialisation
        else:
            for i in range(len(network_layout) -  1):
                input = network_layout[i]
                output = network_layout[i+1]
                # Better He initialization
                weight_init = np.random.randn(output, input) * np.sqrt(2.0 / input)
                bias = np.zeros(output)
                weights.append(weight_init)
                biases.append(bias)

        return weights, biases
    
    
     # Creates empty matrices for weights/bias needed for the optimisers
    def create_zeroed_matrix_per_parameter(self):
        weights_velocity = []
        biases_velocity = []
        for i in range(len(self.weights)):
            weights_velocity.append(np.zeros(self.weights[i].shape))
            biases_velocity.append(np.zeros(self.biases[i].shape))
        return weights_velocity, biases_velocity
    
    
    def train(self, epochs, batch_size=64, monitor=None, lr_decay_factor=0.95, lr_decay_every=10, early_stop_patience=10):
        """
        Execute training loop with optional performance tracking.
        
        Args:
            epochs (int): Total number of training iterations
            batch_size (int): Number of samples per gradient update
            monitor (TrainingMonitor, optional): Callback for tracking metrics
            lr_decay_factor (float): Factor to multiply learning rate by (default: 0.95)
            lr_decay_every (int): Apply decay every N epochs (default: 10)
            early_stop_patience (int): Stop if val loss doesn't improve for N epochs (default: 10)
            
        Returns:
            TrainingMonitor: Object containing complete training statistics
        """
        if monitor is None:
            monitor = TrainingMonitor()
        
        # Early stopping tracking
        best_val_loss = float('inf')
        patience_counter = 0
        initial_lr = self.learning_rate
        
        for epoch_idx in range(epochs):
            # Learning rate decay
            if epoch_idx > 0 and epoch_idx % lr_decay_every == 0:
                self.learning_rate *= lr_decay_factor
                print(f"Learning rate decayed to: {self.learning_rate:.6f}")
            
            # Randomize training order each iteration
            shuffled_x, shuffled_y = self.shuffle_dataset()

            # Initialize epoch statistics
            cumulative_loss = 0.0
            num_batches = 0
            total_correct = 0
            
            # Process mini-batches
            for batch_start in range(0, len(self.inputs), batch_size):
                batch_end = batch_start + batch_size
                x_batch = shuffled_x[batch_start:batch_end]
                y_batch = shuffled_y[batch_start:batch_end]

                # Forward propagation
                _ = self.forwardpass(x_batch, training=True)

                # Calculate loss
                batch_loss = self.cross_entropy_loss(self.softmax.y_pred, y_batch)

                # Backpropagation
                grads_w, grads_b = self.backprop(_, y_batch, batch_size)

                # Parameter update
                self.update_weights(grads_w, grads_b)
                
                # Aggregate batch statistics
                num_batches += 1
                cumulative_loss += batch_loss
                predictions = np.argmax(self.softmax.y_pred, axis=1)
                targets = np.argmax(y_batch, axis=1)
                total_correct += np.sum(predictions == targets)
            
            # Calculate epoch-level metrics
            avg_loss = cumulative_loss / num_batches
            accuracy = total_correct / len(self.inputs)
            
            # Update monitor with training results
            monitor.record_epoch(epoch_idx, avg_loss, accuracy, phase='training')
            
            # Evaluate on hold-out set if available
            if monitor.has_validation_data():
                val_loss, val_accuracy = self._evaluate_performance(monitor.validation_x, monitor.validation_y)
                monitor.record_epoch(epoch_idx, val_loss, val_accuracy, phase='validation')
                
                # Early stopping check
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= early_stop_patience:
                        print(f"\nEarly stopping triggered at epoch {epoch_idx + 1}")
                        print(f"Best validation loss: {best_val_loss:.4f}")
                        break
            
            # Display progress
            monitor.log_progress(epoch_idx, epochs)
        
        return monitor 

    
    def forwardpass(self, batch_x, training=True):
        cache = []
        self.activations[0] = batch_x 
        for j in range(self.hidden_layers + 1):
            
            # Recheck this block and usage of self.dropout_outputs if necessary
            if j==0:
                input_to_layer = self.activations[j]
            else:
                input_to_layer = self.dropout_outputs[j-1]

            
            z1 = np.dot(input_to_layer, self.weights[j].T) + self.biases[j]
                
            # Softmax
            if j == (len(self.weights) - 1):
                self.softmax.forward(z1)
                self.activations[j+1] = self.softmax.y_pred
                cache.append(z1)
                cache.append(self.softmax.y_pred)

            # All other layers
            else:
                a1 = self.activation_func.forward(z1)
                
                # Integrated Dropout
                # Recheck this block and usage of self.dropout_outputs if necessary
                if training:
                    a_dropout = self.dropout_layers[j].forward(a1, training=True)
                else:
                    a_dropout = a1

                self.activations[j+1] = a1
                self.dropout_outputs[j] = a_dropout
                cache.append(z1)
                cache.append(a1)
            
        return cache

    
    def backprop(self, cache, batch_y, batch_size=64):
        weight_gradients = []
        bias_gradients = []
        
        # Softmax layer backprop
        dsMax = self.softmax.backward(batch_y)
        dw_sMax = np.dot(dsMax.T, self.activations[-2])
        db_sMax = np.sum(dsMax, axis=0)

        # Average over batch size
        dw_sMax /= batch_size
        db_sMax /= batch_size

        # Store gradients in weight and bias lists
        weight_gradients.append(dw_sMax)
        bias_gradients.append(db_sMax)
        
        # Running gradient across entire network
        grad = dsMax

        j = len(self.weights) - 2
        for i in range(len(cache) - 4, -1, -2):
            z = cache[i]
            a = cache[i+1]
            prev_a = cache[i-1] if i > 0 else self.activations[0]

            da = np.dot(grad, self.weights[j+1])
            
            # Recheck this block and usage of self.dropout_outputs if necessary
            da = self.dropout_layers[j].backward(da)
            
            dz = da * self.activation_func.backward(z)
            
            dw = np.dot(dz.T, prev_a) / batch_size
            weight_gradients.append(dw)

            db = np.sum(dz, axis=0) / batch_size
            bias_gradients.append(db)
            
            grad = dz
            j -= 1

        weight_gradients.reverse()
        bias_gradients.reverse()
        return weight_gradients, bias_gradients
    
    
    def update_weights(self, weight_gradients, bias_gradients):
        if self.optimiser == "None":
            self.vanille_gradient_descent(weight_gradients, bias_gradients)
        elif self.optimiser == "RMS_Prop":
            self.RMS_Prop(weight_gradients, bias_gradients)
        elif self.optimiser == "Adam":
            self.adam(weight_gradients, bias_gradients)
        else:
            raise ValueError("Optimiser should be either: None, RMS_Prop or Adam")

    
    # Vanilla Mini-Batch GD 
    def vanille_gradient_descent(self, weight_gradients, bias_gradients):
        for i in range(len(weight_gradients)):
            self.weights[i] = self.weights[i] - (self.learning_rate * weight_gradients[i])
            self.biases[i] = self.biases[i] - (self.learning_rate * bias_gradients[i])
        return self.weights

    
    # Optimiser 1 - RMSProp
    # decay rate hyperparam can be tuned but 0.9 is stable
    def RMS_Prop(self, weight_gradients, bias_gradients, decay_rate=0.9, epsilon=1e-8):
        for i in range(len(self.weights)):
            self.r_weights[i] =  (decay_rate * self.r_weights[i]) + ((1 - decay_rate) * (weight_gradients[i] ** 2))
            self.r_biases[i] =  (decay_rate * self.r_biases[i]) + ((1 - decay_rate) * (bias_gradients[i] ** 2))
            self.weights[i] -= (self.learning_rate * weight_gradients[i]) / np.sqrt(self.r_weights[i] + epsilon)
            self.biases[i] -= (self.learning_rate * bias_gradients[i]) / np.sqrt(self.r_biases[i] + epsilon)
        return self.weights

    
    
    # Optimiser 2 - Adam
    def adam(self, weight_gradients, bias_gradients, momentum_Beta=0.9, RMS_Prop_decay=0.999, epsilon=1e-8):
        self.adam_timestep += 1
        # Uses Momentum and RMSProp
        for i in range(len(self.weights)):
            # Moment 1 - Momentum style
            self.adam_moment1_weights[i] = (momentum_Beta * self.adam_moment1_weights[i]) + ((1 - momentum_Beta) * weight_gradients[i])
            self.adam_moment1_biases[i] = (momentum_Beta * self.adam_moment1_biases[i]) + ((1 - momentum_Beta) * bias_gradients[i])
            
            # Moment 2 - RMS_Prop style
            self.adam_moment2_weights[i] = (RMS_Prop_decay * self.adam_moment2_weights[i]) + ((1 - RMS_Prop_decay) * (weight_gradients[i] ** 2))
            self.adam_moment2_biases[i] = (RMS_Prop_decay * self.adam_moment2_biases[i]) + ((1 - RMS_Prop_decay) * (bias_gradients[i] ** 2))
            
            # Use timestep, which is incremented per batch to correct biases
            moment1_weights_bias_corrected = self.adam_moment1_weights[i] / (1 - (momentum_Beta ** self.adam_timestep))
            moment1_biases_bias_corrected = self.adam_moment1_biases[i] / (1 - (momentum_Beta ** self.adam_timestep))
            moment2_weights_bias_corrected = self.adam_moment2_weights[i] / (1 - (RMS_Prop_decay ** self.adam_timestep))
            moment2_biases_bias_corrected = self.adam_moment2_biases[i] / (1 - (RMS_Prop_decay ** self.adam_timestep))
            # Actual update
            self.weights[i] = self.weights[i] - ((self.learning_rate * moment1_weights_bias_corrected) / np.sqrt(moment2_weights_bias_corrected + epsilon))
            self.biases[i] = self.biases[i] - ((self.learning_rate * moment1_biases_bias_corrected) / np.sqrt(moment2_biases_bias_corrected + epsilon))
        return self.weights

    
    
    # Runs the test data (not part of training)
    def run(self, x_test):
        original_inputs = self.inputs
        self.inputs = x_test
        # Ignore cache
        cache = self.forwardpass(x_test, training=False)
        # Creates 1d array where highest probability index = predicted class
        y_pred = np.argmax(self.softmax.y_pred, axis=1)

        self.inputs = original_inputs
        return y_pred

    
    
    def cross_entropy_loss(self, y_pred, y_true, epsilon=1e-8):
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.sum(y_true * np.log(y_pred), axis=1)
        return np.mean(loss)
    
    
    
    # Shuffle dataset for Mini-Batch GD, so it is representative of entire dataset
    def shuffle_dataset(self):
        random_indices = np.random.permutation(len(self.inputs)) # Set of random indices between 0 and end of dataset 
        x_shuffled = self.inputs[random_indices]
        y_shuffled = self.outputs[random_indices]
        return x_shuffled, y_shuffled
    
    
    def _evaluate_performance(self, features, labels):
        """
        Compute loss and accuracy on a given dataset.
        
        Args:
            features (np.ndarray): Input data
            labels (np.ndarray): Ground truth labels (one-hot)
            
        Returns:
            tuple: (loss, accuracy)
        """
        _ = self.forwardpass(features, training=False)
        loss = self.cross_entropy_loss(self.softmax.y_pred, labels)
        predictions = np.argmax(self.softmax.y_pred, axis=1)
        targets = np.argmax(labels, axis=1)
        accuracy = np.mean(predictions == targets)
        return loss, accuracy

In [ ]:
# OPTIMIZED TRAINING CONFIGURATION with improvements
# Partition dataset into training and validation subsets
x_train_subset, y_train_subset, x_val_subset, y_val_subset = create_data_splits(
    x_train, y_train, 
    train_size=35000,  # Training samples (70% of available data)
    val_size=5000      # Validation set
)

print(f"Training Configuration:")
print(f"  Training set: {x_train_subset.shape[0]} samples")
print(f"  Validation set: {x_val_subset.shape[0]} samples")
print(f"  Test set: {x_test.shape[0]} samples")

# Initialize network with optimized settings
np.random.seed(20)
network_fast = NeuralNetwork(
    x_train_subset,
    y_train_subset,
    hidden_layers=4,
    hidden_layer_sizes=[1024, 512, 256, 128],
    activation_func=LeakyReLU(),
    optimiser="Adam",
    learning_rate=0.001,
    dropout_rate=0.05
)

# Create monitor with validation data
performance_tracker_fast = TrainingMonitor(
    validation_x=x_val_subset,
    validation_y=y_val_subset
)

monitor_results_fast = network_fast.train(
    epochs=50,
    batch_size=128,
    monitor=performance_tracker_fast,
    lr_decay_factor=0.95,
    lr_decay_every=10,
    early_stop_patience=999  
)

monitor_results_fast.visualize_performance()
# Visualize results

Training Configuration:
  Training set: 35000 samples
  Validation set: 5000 samples
  Test set: 10000 samples


In [ ]:
# Test the Neural Network (using the trained model)
y_pred = network_fast.run(x_test)
y_true = np.argmax(y_test, axis=1)
# Convert into boolean array and then sum over all true elements
correct_predictions = np.sum((y_pred == y_true))
accuracy = correct_predictions / len(y_true)
print(f"Correct predictions: {correct_predictions} / {len(y_true)}")
print(f"Test Accuracy: {accuracy:.2%}")

Correct predictions: 4735 / 10000
Test Accuracy: 47.35%
